In [ ]:
#Setup and Installation
!nvidia-smi
!git clone --recursive https://github.com/graphdeco-inria/gaussian-splatting.git
%cd gaussian-splatting

!pip install -q plyfile tqdm pillow opencv-python
!pip install -q submodules/diff-gaussian-rasterization
!pip install -q submodules/simple-knn

!apt-get install -y colmap imagemagick ffmpeg

import torch
import os
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

Fri Jan 16 09:32:35 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P0             26W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
pwd

In [ ]:
!ls /kaggle/input/nerf-synthetic-dataset/nerf_synthetic


In [ ]:
!cp -r /kaggle/input/nerf-synthetic-dataset/nerf_synthetic/lego /kaggle/working/lego

In [ ]:
# CELL 3: Convert Dataset
SCENE_NAME = 'lego'
SOURCE_PATH = f'/kaggle/working/{SCENE_NAME}'

!python convert.py -s {SOURCE_PATH}

print(f"Data converted for scene: {SCENE_NAME}")
!ls {SOURCE_PATH}


In [ ]:
# CELL 5: Train Model (Quick - 7000 iterations)
import time

MODEL_PATH = f'output/models/{SCENE_NAME}'
ITERATIONS = 7000

os.makedirs(MODEL_PATH, exist_ok=True)

train_cmd = f"""
python train.py \
    -s {SOURCE_PATH} \
    -m {MODEL_PATH} \
    --iterations {ITERATIONS} \
    --test_iterations 1000 3000 7000 \
    --save_iterations 1000 3000 7000 \
    --checkpoint_iterations 1000 3000 7000 \
    --resolution 1 \
    --eval
"""

print("Starting training...")
start_time = time.time()
!{train_cmd}
end_time = time.time()

print(f"\nTraining completed in {(end_time - start_time) / 60:.2f} minutes")

In [ ]:
# CELL 6: Render Test Views
RENDER_PATH = f'output/renders/{SCENE_NAME}'

render_cmd = f"""
python render.py \
    -m {MODEL_PATH} \
    --skip_train \
    --iteration 7000
"""

print("Rendering test views...")
!{render_cmd}

print(f"Renders saved to: {MODEL_PATH}/test/ours_7000/renders")

In [ ]:
# CELL 7: Evaluate Model
eval_cmd = f"""
python metrics.py \
    -m {MODEL_PATH}
"""

print("Evaluating model...")
!{eval_cmd}

In [ ]:
# CELL 8: Visualize Results
import json
import matplotlib.pyplot as plt
from PIL import Image

renders_path = f"{MODEL_PATH}/test/ours_7000/renders"
gt_path = f"{MODEL_PATH}/test/ours_7000/gt"

render_files = sorted(os.listdir(renders_path))[:8]

fig, axes = plt.subplots(4, 4, figsize=(16, 16))

for idx, img_file in enumerate(render_files):
    render_img = Image.open(os.path.join(renders_path, img_file))
    gt_img = Image.open(os.path.join(gt_path, img_file))
    
    axes[idx*2//4, (idx*2)%4].imshow(render_img)
    axes[idx*2//4, (idx*2)%4].set_title(f'Rendered {idx}')
    axes[idx*2//4, (idx*2)%4].axis('off')
    
    axes[idx*2//4, (idx*2)%4 + 1].imshow(gt_img)
    axes[idx*2//4, (idx*2)%4 + 1].set_title(f'Ground Truth {idx}')
    axes[idx*2//4, (idx*2)%4 + 1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# # CELL 9: Create Video from Renders
# video_path = f"output/videos/{SCENE_NAME}_test.mp4"
# os.makedirs(os.path.dirname(video_path), exist_ok=True)

# ffmpeg_cmd = f"""
# ffmpeg -y -framerate 30 -pattern_type glob -i '{renders_path}/*.png' \
#     -c:v libx264 -pix_fmt yuv420p {video_path}
# """

# !{ffmpeg_cmd}

# print(f"Video saved to: {video_path}")

# from IPython.display import Video
# Video(video_path, width=800)

In [ ]:
# CELL 10: Load Metrics
results_file = f"{MODEL_PATH}/results.json"

if os.path.exists(results_file):
    with open(results_file, 'r') as f:
        results = json.load(f)
    
    print("Evaluation Metrics:")
    print("=" * 50)
    for key, value in results.items():
        if isinstance(value, dict):
            print(f"\n{key}:")
            for k, v in value.items():
                print(f"  {k}: {v}")
        else:
            print(f"{key}: {value}")
    print("=" * 50)
else:
    print("Results file not found. Run evaluation first.")

In [ ]:
# CELL 11: Render Custom Camera Path (360 rotation)
custom_render_path = f"output/renders/{SCENE_NAME}_360"
os.makedirs(custom_render_path, exist_ok=True)

render_360_cmd = f"""
python render.py \
    -m {MODEL_PATH} \
    --iteration 7000 \
    --skip_train \
    --skip_test
"""

!{render_360_cmd}

print("Custom camera path rendered")

In [ ]:
# CELL 12: Train Full Quality (Optional - 30000 iterations)
FULL_MODEL_PATH = f'output/models/{SCENE_NAME}_full'
FULL_ITERATIONS = 30000

os.makedirs(FULL_MODEL_PATH, exist_ok=True)

full_train_cmd = f"""
python train.py \
    -s {SOURCE_PATH} \
    -m {FULL_MODEL_PATH} \
    --iterations {FULL_ITERATIONS} \
    --test_iterations 7000 15000 30000 \
    --save_iterations 7000 15000 30000 \
    --checkpoint_iterations 7000 15000 30000 \
    --resolution 1 \
    --eval
"""

print("Starting full training (30000 iterations)...")
print("This will take 20-30 minutes")

start_time = time.time()
!{full_train_cmd}
end_time = time.time()

print(f"\nFull training completed in {(end_time - start_time) / 60:.2f} minutes")

In [ ]:
# CELL 13: Process Custom Images (Optional)
CUSTOM_PATH = "custom-scene/custom_scene"

os.makedirs(f"{CUSTOM_PATH}/input", exist_ok=True)

print("Upload your images to /kaggle/input/ through Kaggle UI")
print("Then run:")
print(f"!cp /kaggle/input/your-dataset/*.jpg {CUSTOM_PATH}/input/")
print(f"!python convert.py -s {CUSTOM_PATH} --colmap_executable colmap")

In [ ]:
!cp /kaggle/input/custom-scene/custom_scene/*.jpg {CUSTOM_PATH}/input/

In [ ]:
# # CELL 14: Process Video to Images (Optional)
# VIDEO_PATH = "data/custom_video/video.mp4"
# VIDEO_OUTPUT = "data/custom_video/images"

# os.makedirs(VIDEO_OUTPUT, exist_ok=True)

# video_to_frames_cmd = f"""
# ffmpeg -i {VIDEO_PATH} -qscale:v 1 -qmin 1 -vf fps=2 {VIDEO_OUTPUT}/%04d.jpg
# """

# print("Upload your video first, then run:")
# print(video_to_frames_cmd)

In [ ]:
# CELL 15: Export Point Cloud
ply_path = f"{MODEL_PATH}/point_cloud/iteration_7000/point_cloud.ply"

if os.path.exists(ply_path):
    print(f"Point cloud saved at: {ply_path}")
    
    from plyfile import PlyData
    plydata = PlyData.read(ply_path)
    
    print(f"Number of points: {plydata['vertex'].count}")
    print(f"Properties: {plydata['vertex'].properties}")
else:
    print("Point cloud not found")

In [ ]:
# CELL 16: Cleanup and Save Results
import shutil

print("Saving important files...")

files_to_keep = [
    f"{MODEL_PATH}/point_cloud/iteration_7000/point_cloud.ply",
    f"{MODEL_PATH}/cameras.json",
    f"{MODEL_PATH}/results.json",
]

for file_path in files_to_keep:
    if os.path.exists(file_path):
        print(f"✓ {file_path}")
    else:
        print(f"✗ {file_path} not found")

print("\nModel training complete!")
print(f"Model saved at: {MODEL_PATH}")
print(f"Renders saved at: {MODEL_PATH}/test/ours_7000/renders")

In [ ]:
# CELL 17: Compare Multiple Scenes
SCENES = ['lego', 'chair', 'drums']

for scene in SCENES:
    source = f'data/nerf_synthetic/{scene}'
    model = f'output/models/{scene}_quick'
    
    print(f"\nProcessing {scene}...")
    !python convert.py -s {source}
    !python train.py -s {source} -m {model} --iterations 5000 --resolution 2 --eval
    !python render.py -m {model} --skip_train --iteration 5000
    !python metrics.py -m {model}

In [ ]:
# CELL 18: Advanced Training Options
ADVANCED_MODEL_PATH = f'output/models/{SCENE_NAME}_advanced'

advanced_train_cmd = f"""
python train.py \
    -s {SOURCE_PATH} \
    -m {ADVANCED_MODEL_PATH} \
    --iterations 30000 \
    --position_lr_init 0.00016 \
    --position_lr_final 0.0000016 \
    --feature_lr 0.0025 \
    --opacity_lr 0.05 \
    --scaling_lr 0.005 \
    --rotation_lr 0.001 \
    --densify_grad_threshold 0.0002 \
    --densification_interval 100 \
    --opacity_reset_interval 3000 \
    --densify_from_iter 500 \
    --densify_until_iter 15000 \
    --sh_degree 3 \
    --eval
"""

print("Training with custom hyperparameters...")
!{advanced_train_cmd}

In [ ]:
# CELL 19: Batch Render All Test Views
all_renders_path = f"output/all_renders/{SCENE_NAME}"
os.makedirs(all_renders_path, exist_ok=True)

iterations = [1000, 3000, 7000]

for iter_num in iterations:
    print(f"\nRendering iteration {iter_num}...")
    render_cmd = f"""
    python render.py \
        -m {MODEL_PATH} \
        --iteration {iter_num} \
        --skip_train
    """
    !{render_cmd}
    
    src = f"{MODEL_PATH}/test/ours_{iter_num}/renders"
    dst = f"{all_renders_path}/iter_{iter_num}"
    
    if os.path.exists(src):
        shutil.copytree(src, dst, dirs_exist_ok=True)
        print(f"Saved to {dst}")

In [ ]:
# CELL 20: Final Summary
print("=" * 80)
print("TRAINING SUMMARY")
print("=" * 80)
print(f"Scene: {SCENE_NAME}")
print(f"Model Path: {MODEL_PATH}")
print(f"Iterations: {ITERATIONS}")
print()

if os.path.exists(f"{MODEL_PATH}/results.json"):
    with open(f"{MODEL_PATH}/results.json", 'r') as f:
        results = json.load(f)
    print("Metrics:")
    for key, value in results.items():
        if not isinstance(value, dict):
            print(f"  {key}: {value}")

print()
print("Output Files:")
print(f"  - Model: {MODEL_PATH}")
print(f"  - Renders: {MODEL_PATH}/test/ours_7000/renders")
print(f"  - Point Cloud: {MODEL_PATH}/point_cloud/iteration_7000/point_cloud.ply")
print("=" * 80)